In [1]:
from cerebro.backend import SparkBackend
from cerebro.keras import SparkEstimator

# datas storage for intermediate data and model artifacts.
from cerebro.storage import LocalStore, HDFSStore

# Model selection/AutoML methods.
from cerebro.tune import GridSearch, RandomSearch, TPESearch

# Utility functions for specifying the search space.
from cerebro.tune import hp_choice, hp_uniform, hp_quniform, hp_loguniform, hp_qloguniform

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pyspark.sql import SparkSession
import numpy as np


spark = SparkSession \
    .builder \
    .appName("Cerebro Example") \
    .getOrCreate()

...
work_dir = '/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/'
backend = SparkBackend(spark_context=spark.sparkContext, num_workers=1)
store = LocalStore(prefix_path=work_dir + 'test/')

df = spark.read.format("libsvm") \
    .option("numFeatures", "784") \
    .load("/Users/zijian/Desktop/ucsd/cse234/project/mnist/mnist.scale")

21/12/02 04:07:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


CEREBRO => Time: 2021-12-02 04:07:12, Running 1 Workers


In [2]:
df.count()
df = df.limit(100)

In [3]:
from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(dropLast=False)
encoder.setInputCols(["label"])
encoder.setOutputCols(["label_OHE"])

encoder_model = encoder.fit(df)
encoded = encoder_model.transform(df)

feature_columns=['features']
label_columns=['label_OHE']
train_df, test_df = encoded.randomSplit([0.8, 0.2], seed=100)

In [4]:
from keras_tuner.engine import hyperparameters
import autokeras as ak
from cerebro.nas.hphpmodel import HyperHyperModel

img_shape = (28, 28, 1)

input_node = ak.ImageInput()
output_node = ak.ConvBlock(
    kernel_size=hyperparameters.Fixed('kernel_size', value=3),
    num_blocks=hyperparameters.Fixed('num_blocks', value=1),
    num_layers=hyperparameters.Fixed('num_layers', value=2),
)(input_node)
output_node = ak.ClassificationHead()(output_node)
am = HyperHyperModel(input_node, output_node, seed=2000)

am.resource_bind(
    backend=backend, 
    store=store,
    feature_columns=feature_columns,
    label_columns=label_columns,
    evaluation_metric='accuracy', 
)

am.tuner_bind(
    tuner="greedy", 
    hyperparameters=None, 
    objective="val_accuracy",
    max_trials=1,
    overwrite=True,
    exploration=0.3,
)

In [5]:
rel = am.fit(train_df, epochs=1, input_shape=img_shape)

Trial 1 Complete [00h 00m 09s]
val_accuracy: 0.0625

Best val_accuracy So Far: 0.0625
Total elapsed time: 00h 00m 09s


In [6]:
with open("mnist_nas_logs.txt", "w") as file:
    file.writelines(rel.metrics)

In [7]:
rel.metrics

{'model_0_1638446850': {'trial': <keras_tuner.engine.trial.Trial at 0x1830bdfd0>,
  'train_loss': [2.302694082260132],
  'train_accuracy': [0.09375],
  'val_loss': [2.3033695220947266],
  'val_accuracy': [0.0625]}}

[Stage 15:>                                                         (0 + 1) / 1]

In [13]:
import json
m = {}
for model in rel.metrics:
    m[model] = {}
    for key in rel.metrics[model]:
        if key != 'trial':
            m[model][key] = rel.metrics[model][key]
with open("mnist_nas_logs.txt", "w") as file:
    file.write(json.dumps(m))